## Engenharia de características

#### Identificando as Variáveis

Aqui as váriáveis que serão processadas pelos algorítmos são produzidas e identificadas.

A codificação das colunas categóricas será executada.

In [ ]:
import pandas as pd

In [ ]:
# Identifica a primeira e a última data no histórico de importação, para serem utilizadas
# na na eliminação do efeito colateral***
pr_d=i_o[['ANO','MES']].head(1).astype(int)
ut_d=i_o[['ANO','MES']].tail(1).astype(int)

*** Efetito colateral é a criação de meses antes do início real e após o último mês, no primeiro e no ano corrente, respectivamente, após o processo de agrupamento que consideram ANO e MES como tipo 'category'.

In [ ]:
# Idenifica as colunas categoricas
i[['ANO','MES','NCM']]=i[['ANO','MES','NCM']].astype('category')


# procede o agrupamento categórico transformando i em matriz densa
i=i.\
groupby(by=['ANO','MES','NCM'])\
[i.select_dtypes(include='float64').columns.tolist()]\
.sum().reset_index()


In [ ]:
i['date']=pd.to_datetime(i.ANO.astype(str)+'/'+i.MES.astype(str)+'/1').astype(int)

In [ ]:
# Elimina o efeito colateral***
i=i.loc[(i.ANO.astype(int)>pr_d.ANO.values[0])|(i.MES.astype(int)>=pr_d.MES.values[0])]
i=i.loc[(i.ANO.astype(int)<ut_d.ANO.values[0])|(i.MES.astype(int)<=ut_d.MES.values[0])]

In [ ]:
# Cria-se a coluna que classifica se a importação ocorreu ou não 
# Inicialmente, todas importações são marcadas como zero (**nâo ocorridas**)
i['oc']=0
# Corrigi-se a coluna 'oc', passando a marcar '1' para as coluna que apresentam alguma quantidade importada
i.loc[i.KG>0,'oc']=1

In [ ]:
X_classif=i.drop(columns=['ANO','oc','KG'])
y_classif=i[['oc']]

X_regress=i[i.KG>0].drop(columns=['ANO','oc','KG'])
y_regress=i[i.KG>0][['KG']]

In [ ]:
X_classif_train_test, X_classif_test, y_classif_train_test, y_classif_test=\
tts(X_classif,y_classif,train_size=.7)

In [ ]:
X_regress_train_test, X_regress_test, y_regress_train_test, y_regress_test=\
tts(X_regress,y_regress,train_size=.7)

In [ ]:
# Ajuntando dados de treino e teste, assinalando um e outro para separação posterior
X_classif_train_test_test_to_dumm=pd.concat([X_classif_train_test.assign(train=1),X_classif_test.assign(train=0)])

# Codificando treino e teste de uma só ves
X_classif_train_test_test_dumm=pd.get_dummies(X_classif_train_test_test_to_dumm)

# Separando treino de teste com categorias codificadas
X_classif_train_test_dumm=X_classif_train_test_test_dumm[X_classif_train_test_test_dumm.train==1]\
.drop(columns='train')
X_classif_test_dumm=X_classif_train_test_test_dumm[X_classif_train_test_test_dumm.train==0]\
.drop(columns='train')

In [ ]:
# Ajuntando dados de treino e teste, assinalando um e outro para separação posterior
X_regress_train_test_test_to_dumm=pd.concat([X_regress_train_test.assign(train=1),X_regress_test.assign(train=0)])

# Codificando treino e teste de uma só ves
X_regress_train_test_test_dumm=pd.get_dummies(X_regress_train_test_test_to_dumm)

# Separando treino de teste com categorias codificadas
X_regress_train_test_dumm=X_regress_train_test_test_dumm[X_regress_train_test_test_dumm.train==1]\
.drop(columns='train')
X_regress_test_dumm=X_regress_train_test_test_dumm[X_regress_train_test_test_dumm.train==0]\
.drop(columns='train')

##### 1.3- Variáveis para pred

1.3.1- Identificando as variáveis de treino para previsão

In [ ]:
X_classif_train_pred=X_classif.copy()
y_classif_train_pred=y_classif.copy()

In [ ]:
X_regress_train_pred=X_regress.copy()
y_regress_train_pred=y_regress.copy()

1.3.2- Identificando as variáveis para previsão

In [ ]:
# Identificando a data (d_pred) para previsão, que no caso, é um mês a mais do que o último
# mês do histórico de importação
d_pred_t=ut_d.copy()
d_pred_t.MES=d_pred_t.MES+1

d_pred=d_pred_t.copy()

In [ ]:
# Combinando as demais colunas, que neste caso é somente o NCM,com a data a ser prevista
X_classif_pred=d_pred.assign(key=0)\
.merge(i[['NCM']]\
       .astype('category')\
       .drop_duplicates()\
       .assign(key=0))\
.drop(columns='key')

# Cria date
X_classif_pred=X_classif_pred.assign(
    date=
    pd.to_datetime(
        X_classif_pred.ANO.astype(str)
        +'/'+
        X_classif_pred.MES.astype(str))
    .astype(int))

# Deleta ANO
X_classif_pred=X_classif_pred.drop(columns='ANO')

i_agg=pd.read_csv('~/comex/data/i_agg.csv')

if agg==1:

    # Prevê que o input
    X_classif_pred['k1']=i_agg.k1.iloc[-1]
    X_classif_pred['k2']=i_agg.k2.iloc[-1]
    X_classif_pred['k3']=i_agg.k3.iloc[-1]
    X_classif_pred['k4']=i_agg.k4.iloc[-1]
    X_classif_pred['k5']=i_agg.k5.iloc[-1]
    X_classif_pred['k6']=i_agg.k6.iloc[-1]

    
#display(X_classif_pred.sk)
# Ajuntando dados de treino e pred, assinalando um e outro para separação posterior

Dumm

In [ ]:
X_classif_train_pred_pred_to_dumm=\
pd.concat([
    X_classif_train_pred.assign(train=1),
    X_classif_pred      .assign(train=0)
])

# Identificando as colunas categóricas para a seguir serem também codificadas, como na fase de test
X_classif_train_pred_pred_to_dumm[['MES','NCM']]=X_classif_train_pred_pred_to_dumm[['MES','NCM']].astype('category')

# Codificando treino e pred de uma só ves
X_classif_train_pred_pred_dumm=pd.get_dummies(X_classif_train_pred_pred_to_dumm)

# Separando treino de pred com categorias codificadas
X_classif_train_pred_dumm=X_classif_train_pred_pred_dumm[X_classif_train_pred_pred_dumm.train==1]\
.drop(columns='train')

X_classif_pred_dumm=X_classif_train_pred_pred_dumm[X_classif_train_pred_pred_dumm.train==0]\
.drop(columns='train')

In [ ]:
# Copiando todo X de classif para regress
i_classif_train_pred_dumm=pd.concat([X_classif_train_pred_dumm,y_classif_train_pred],axis=1)

i_regress_train_pred_dumm=i_classif_train_pred_dumm[i_classif_train_pred_dumm.oc==1]

X_regress_train_pred_dumm=i_regress_train_pred_dumm.drop(columns='oc')

In [ ]:
# Copiando todo X de classif para regress
X_regress_pred=X_classif_pred
X_regress_pred_dumm=X_classif_pred_dumm